# Import Library

In [4]:
import numpy as np
import pandas as pd
import os

# Import Data

In [5]:
df = pd.read_csv(os.path.join(os.getcwd(), "..", "data", "Amazon-Products.csv"))
df.head()

,Unnamed: 0,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990"
1,1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990"
2,2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990"
3,3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990"
4,4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790"


In [6]:
df.shape

(551585, 10)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551585 entries, 0 to 551584
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      551585 non-null  int64 
 1   name            551585 non-null  object
 2   main_category   551585 non-null  object
 3   sub_category    551585 non-null  object
 4   image           551585 non-null  object
 5   link            551585 non-null  object
 6   ratings         375791 non-null  object
 7   no_of_ratings   375791 non-null  object
 8   discount_price  490422 non-null  object
 9   actual_price    533772 non-null  object
dtypes: int64(1), object(9)
memory usage: 42.1+ MB


# Preprocessing

In [8]:
# Drop unnecessary columns
df.drop(columns=['Unnamed: 0', 'image', 'link'], inplace=True)

In [9]:
print(f"Total null value in product table:\n{df.isnull().sum()}")

Total null value in product table:
name                   0
main_category          0
sub_category           0
ratings           175794
no_of_ratings     175794
discount_price     61163
actual_price       17813
dtype: int64


## Handle missing values

In [10]:
# FIll with 0
df['ratings'] = df['ratings'].fillna(0)
df['no_of_ratings'] = df['no_of_ratings'].fillna(0)

# No Discount
df['discount_price'] = df['discount_price'].fillna(df['actual_price'])

# Drop because no price
df = df.dropna(subset=['actual_price'])

## Check unique value

In [11]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Column: {column}")
    print(f"Unique Values: {unique_values}\n")

Column: name
Unique Values: ['Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1 Convertible, Copper, Anti-Viral + Pm 2.5 Filter, 2023 Model, White, Gls18I3...'
 'LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...'
 'LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Copper, Super Convertible 6-In-1 Cooling, Hd Filter With Anti Virus Protection,...'
 ... 'Redwolf Noice Toit Smort - Hoodie (Black)'
 'Redwolf Schrute Farms B&B - Hoodie (Navy Blue)'
 'Mothercare Printed Cotton Elastane Girls Infant Leggings (S21VF342MU-P)']

Column: main_category
Unique Values: ['appliances' 'car & motorbike' 'tv, audio & cameras' 'sports & fitness'
 'grocery & gourmet foods' 'home & kitchen' 'pet supplies' 'stores'
 'toys & baby products' "kids' fashion" 'bags & luggage' 'accessories'
 "women's shoes" 'beauty & health' "men's shoes" "women's clothing"
 'industrial supplies' "men's clothing" 'music' 'home, kitchen, pets']

Column: sub_category

## Clean name

In [12]:
# Remove Unnecessary Special Characters
df['name'] = df['name'].str.replace(r"[^\w\s.-]", "", regex=True)

# Standardize Spaces
df['name'] = df['name'].str.replace(r"\s+", " ", regex=True).str.strip()

# Lower character
df['name'] = df['name'].str.lower()

## Clean main_category

In [13]:
# Strip spaces and standardize case
df['main_category'] = df['main_category'].str.strip().str.lower()

# Replace special characters
df['main_category'] = df['main_category'].str.replace(r'&', 'and', regex=True)
df['main_category'] = df['main_category'].str.replace(r"[^\w\s&-]", "", regex=True)  

## Clean sub_category

In [14]:
# Strip spaces and standardize case (optional)
df['sub_category'] = df['sub_category'].str.strip().str.lower()

# Remove special characters
df['sub_category'] = df['sub_category'].str.replace(r'&', 'and', regex=True)
df['sub_category'] = df['sub_category'].str.replace(r"[^\w\s&-]", '', regex=True)  

# Replace unwanted prefixes
df['sub_category'] = df['sub_category'].str.replace(r'^all ', '', regex=True)

## Clean ratings

In [15]:
# Function to clean ratings
def clean_ratings(value):
    value = str(value)  # Ensure it's a string
    
    if any(char in value for char in ['₹', '$', '€', 'FREE', 'Get']):  
        return 0  # Set any value with symbols or invalid words to 0
    
    # Keep only numbers and decimals
    cleaned_value = ''.join(c for c in value if c.isdigit() or c == '.')
    
    return float(cleaned_value) if cleaned_value else 0  # Convert to float, replace empty with 0

# Apply cleaning function to the column
df['ratings'] = df['ratings'].apply(clean_ratings)

## Clean no_of_ratings column

In [16]:
df['no_of_ratings'] = df['no_of_ratings'].astype(str).str.replace(r'[^\d.]', '', regex=True)
df['no_of_ratings'] = df['no_of_ratings'].replace('', '0')
df['no_of_ratings'] = df['no_of_ratings'].astype(float).fillna(0).astype(int)

## Clean discount_price

In [17]:
df['discount_price'] = df['discount_price'].astype(str).str.replace(r'[^\d.]', '', regex=True)
df['discount_price'] = df['discount_price'].astype(float)

## Clean actual_price

In [18]:
df['actual_price'] = df['actual_price'].astype(str).str.replace(r'[^\d.]', '', regex=True)
df['actual_price'] = df['actual_price'].astype(float)

# Save cleaned data

In [19]:
cleaned_file_path = os.path.join(os.getcwd(), "..", "data", "Amazon-Products-Cleaned.csv")
df.to_csv(cleaned_file_path, index=False)